## Imports

In [2]:
import pandas as pd
from typing import *
from datetime import datetime, timedelta

## Functions

In [3]:
def to_dates(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """Gets dataframe and specific columns and convert the columns type to DateTime"""
    for col in cols:
        df[col] = pd.to_datetime(df[col], format='mixed')
    return df

In [4]:
def filter_table(df: pd.DataFrame, time_col: str, drugs_or_diagnosis: bool=False) -> pd.DataFrame:
    """filter the table according the admission time, in order to remain with the measurements from the first 24h"""
    df = pd.merge(dates_labeled_df[['case_no', 'admission_time']], df, on=['case_no'], how='left')
    if not drugs_or_diagnosis:
        df['time_from_admission'] = df[time_col] - df['admission_time']
        one_day = timedelta(days=1)
        df = df.loc[df['time_from_admission'] <= one_day]
    else:
        df = df.loc[df[time_col] < df['admission_time'].dt.date]
    return df

## Read data

In [4]:
aki_admission_df = pd.read_csv('labels/aki_admission.csv', index_col=0)
no_aki_admission_df = pd.read_csv('labels/no_aki_admission.csv', index_col=0)

## Concat tables and add label

In [5]:
aki_admission_df['label'] = 1
no_aki_admission_df['label'] = 0

In [ ]:
dates_labeled_df = pd.concat([aki_admission_df, no_aki_admission_df])
dates_labeled_df = to_dates(dates_labeled_df, ['admission_time'])

In [7]:
dates_labeled_df['label'].value_counts()

label
0    11878
1     4608
Name: count, dtype: int64

## Physical tests Jugular Vein

In [5]:
jugular_vein_df = pd.read_csv('data/Physical_Tests_JugularVeins.csv')
jugular_vein_df = to_dates(jugular_vein_df, ['PhysicalTests_TIME'])

In [18]:
filtered_jugular_df = filter_table(jugular_vein_df, 'PhysicalTests_TIME')

## Physical tests limbs

In [19]:
limbs_df = pd.read_csv('data/Physical_Tests_Limbs.csv')
limbs_df = to_dates(limbs_df, ['PhysicalTests_TIME'])

In [21]:
filtered_limbs_df = filter_table(limbs_df, 'PhysicalTests_TIME')

## Physical tests lungs

In [22]:
lungs_df = pd.read_csv('data/Physical_Tests_Lungs.csv')
lungs_df = to_dates(lungs_df, ['PhysicalTests_TIME'])

In [24]:
filtered_lungs_df = filter_table(lungs_df, 'PhysicalTests_TIME')

## Measures

In [25]:
measures_df = pd.read_csv('data/measures.csv')
measures_df = to_dates(measures_df, ['MEASURE_DATE', 'MEASURE_TIME'])

/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_752/2719536068.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  measures_df = pd.read_csv('data/measures.csv')


In [27]:
# clear rows with missing date
measures_df = measures_df.loc[measures_df['MEASURE_DATE'].notna()]

In [28]:
filtered_measures_df = filter_table(measures_df, 'MEASURE_TIME')

## Diagnosis

In [29]:
diagnosis_df = pd.read_csv('data/diags.csv')
diagnosis_df = to_dates(diagnosis_df, ['DIAG_DATE'])

In [31]:
# clear rows with missing dates
diagnosis_df = diagnosis_df.loc[diagnosis_df['DIAG_DATE'].notna()]

In [32]:
# take diagnosis before the admission date
filtered_diagnosis_df = filter_table(diagnosis_df, 'DIAG_DATE', True)

## Drugs

In [33]:
drug_adm_df = pd.read_csv('data/drugs_adm.csv')
drug_adm_df.rename(columns={'CASE_NO': 'case_no'}, inplace=True)
drug_adm_df = to_dates(drug_adm_df, ['RECOMMENDATION_TIME', 'DRUG_START_TIME'])

In [35]:
# clear rows with missing dates
drug_adm_df = drug_adm_df.loc[drug_adm_df['DRUG_START_TIME'].notna()]

In [36]:
# take drugs before the admission date
filtered_drug_adm_df = filter_table(drug_adm_df, 'DRUG_START_TIME', True)

In [37]:
drug_med_df = pd.read_csv('data/drugs_med.csv')
drug_med_df.rename(columns={'CASE_NO': 'case_no'}, inplace=True)
drug_med_df = to_dates(drug_med_df, ['RECOMMENDATION_TIME', 'DRUG_START_TIME'])

In [39]:
# clear rows with missing dates
drug_med_df = drug_med_df.loc[drug_med_df['DRUG_START_TIME'].notna()]

In [40]:
# take drugs before the admission date
filtered_drug_med_df = filter_table(drug_med_df, 'DRUG_START_TIME', True)

In [41]:
drug_recom_df = pd.read_csv('data/drugs_recom.csv')
drug_recom_df.rename(columns={'CASE_NO': 'case_no'}, inplace=True)
drug_recom_df = to_dates(drug_recom_df, ['RECOMMENDATION_TIME'])

In [43]:
# clear rows with missing dates
drug_recom_df = drug_recom_df.loc[drug_recom_df['RECOMMENDATION_TIME'].notna()]

In [44]:
# take drugs before the admission date
filtered_drug_recom_df = filter_table(drug_recom_df, 'RECOMMENDATION_TIME', True)

## wards

In [46]:
wards_df = pd.read_csv('data/wards.csv', parse_dates=['dept_date_in', 'dept_date_out'])
wards_df = to_dates(wards_df, ['dept_date_in', 'dept_date_out'])

## labs

In [48]:
labs_df = pd.read_csv('all_labs.csv', index_col=0)
labs_df = to_dates(labs_df, ['LAB_TIME'])

/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_752/648249376.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs_df = pd.read_csv('all_labs.csv', index_col=0)


In [50]:
# clear rows with missing dates
labs_df = labs_df.loc[labs_df['LAB_DATE'].notna()]

In [ ]:
# take drugs before the admission date
filtered_labs_df = filter_table(labs_df, 'LAB_TIME')
# remove the objective lab test
filtered_labs_df = filtered_labs_df.loc[filtered_labs_df['LAB_EXAMINATION_NAME'] != 'Creatinine-BL']

## Save tables

In [52]:
filtered_jugular_df.to_csv('filtered data/filtered_jugular_df.csv')
filtered_limbs_df.to_csv('filtered data/filtered_limbs_df.csv')
filtered_lungs_df.to_csv('filtered data/filtered_lungs_df.csv')
filtered_diagnosis_df.to_csv('filtered data/filtered_diagnosis_df.csv')
filtered_measures_df.to_csv('filtered data/filtered_measures_df.csv')
filtered_labs_df.to_csv('filtered data/filtered_labs_df.csv')
filtered_drug_adm_df.to_csv('filtered data/filtered_drug_adm_df.csv')
filtered_drug_med_df.to_csv('filtered data/filtered_drug_med_df.csv')
filtered_drug_recom_df.to_csv('filtered data/filtered_drug_recom_df.csv')